In [1]:
import pandas as pd
from utilities import get_banks, get_hashtags, generate_df_hashtags, generate_df_tweet_words

In [2]:
# load full_analysis.parquet
df = pd.read_parquet('Analysis_results/full_analysis.parquet')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4469911 entries, 0 to 4469910
Data columns (total 13 columns):
 #   Column         Dtype              
---  ------         -----              
 0   Datetime       datetime64[ns, UTC]
 1   Tweet_Id       int64              
 2   Tweet          string             
 3   Username       string             
 4   Reply_Count    int64              
 5   Retweet_Count  int64              
 6   Like_Count     int64              
 7   Bank           string             
 8   Tweet_Clean    object             
 9   Sentiment      object             
 10  Hate_Speech    object             
 11  Offensive      object             
 12  Topic_Sport    object             
dtypes: datetime64[ns, UTC](1), int64(4), object(5), string(3)
memory usage: 443.3+ MB


In [4]:
# drop duplicates based on Tweet_Id
df.drop_duplicates(subset=['Tweet_Id'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4321871 entries, 0 to 4469910
Data columns (total 13 columns):
 #   Column         Dtype              
---  ------         -----              
 0   Datetime       datetime64[ns, UTC]
 1   Tweet_Id       int64              
 2   Tweet          string             
 3   Username       string             
 4   Reply_Count    int64              
 5   Retweet_Count  int64              
 6   Like_Count     int64              
 7   Bank           string             
 8   Tweet_Clean    object             
 9   Sentiment      object             
 10  Hate_Speech    object             
 11  Offensive      object             
 12  Topic_Sport    object             
dtypes: datetime64[ns, UTC](1), int64(4), object(5), string(3)
memory usage: 461.6+ MB


In [5]:
# rename 'Bank' column to 'Bank_Search'
df.rename(columns={'Bank': 'Bank_Search'}, inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4321871 entries, 0 to 4469910
Data columns (total 13 columns):
 #   Column         Dtype              
---  ------         -----              
 0   Datetime       datetime64[ns, UTC]
 1   Tweet_Id       int64              
 2   Tweet          string             
 3   Username       string             
 4   Reply_Count    int64              
 5   Retweet_Count  int64              
 6   Like_Count     int64              
 7   Bank_Search    string             
 8   Tweet_Clean    object             
 9   Sentiment      object             
 10  Hate_Speech    object             
 11  Offensive      object             
 12  Topic_Sport    object             
dtypes: datetime64[ns, UTC](1), int64(4), object(5), string(3)
memory usage: 461.6+ MB


In [7]:
# get banks from tweet
df['Bank'] = df['Tweet_Clean'].apply(get_banks)

In [8]:
df.head()

,Datetime,Tweet_Id,Tweet,Username,Reply_Count,Retweet_Count,Like_Count,Bank_Search,Tweet_Clean,Sentiment,Hate_Speech,Offensive,Topic_Sport,Bank
0,2011-12-31 23:48:36+00:00,153261300485591041,When I went to lunch with @da_realdeal_fnb ht...,sheaintPHEB,0,0,0,fnb,when i went to lunch with,neutral,non-hate,non-offensive,daily_life,[]
1,2011-12-31 23:19:53+00:00,153254073477513216,RT @Naaomsss: HAPPY NEW YEAR MY LOVELY SCHOOLM...,Verwijderddt,0,0,0,fnb,happy new year my lovely schoolmates,positive,non-hate,non-offensive,daily_life,[]
2,2011-12-31 21:52:41+00:00,153232131773038592,"#thinking bout her,da way she makez me feel ho...",Cormaine,1,0,0,fnb,bout herda way she makez me feel how i feel bo...,positive,non-hate,non-offensive,daily_life,[]
3,2011-12-31 21:04:01+00:00,153219883688136704,@alphonsines don't like to work in hotel/fnb i...,whacky_whacky,1,0,0,fnb,dont like to work in hotelfnb industry eh haha,negative,non-hate,non-offensive,pop_culture,[fnb]
4,2011-12-31 20:00:10+00:00,153203815078825984,@ncallegari I just got my upgrade 3 months ago...,tamiyastone,0,0,0,fnb,i just got my upgrade months ago so as soon as...,positive,non-hate,non-offensive,science_&_technology,[]


In [9]:
# if the length of Bank is 0, then use the [bank_search in an array
df['Bank'] = df.apply(lambda x: [x['Bank_Search']] if len(x['Bank']) == 0 else x['Bank'], axis=1)

In [10]:
# print the length of Bank for each tweet
df['Bank'].str.len().value_counts()

1    4248496
2      61579
3       7954
4       2769
5       1065
6          6
7          2
Name: Bank, dtype: int64

In [11]:
# if the length of Bank is greater than 5, then set the Bank to a link of unique banks
bank_list = ['fnb', 'absa', 'nedbank', 'capitec', 'standard_bank']
df.loc[df['Bank'].str.len() > 5, 'Bank'] = df['Bank'].apply(lambda x: list(set(x) & set(bank_list)))

In [12]:
# print tweets where length of Bank is greater than 5
df.loc[df['Bank'].str.len() > 5, 'Bank']

Series([], Name: Bank, dtype: object)

In [13]:
# print unique values of Bank
df['Bank'].explode().unique()

array(['fnb', 'absa', 'standard_bank', 'nedbank', 'capitec'], dtype=object)

In [14]:
# for each bank in Bank, create a new row
df = df.explode('Bank')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4411950 entries, 0 to 4469910
Data columns (total 14 columns):
 #   Column         Dtype              
---  ------         -----              
 0   Datetime       datetime64[ns, UTC]
 1   Tweet_Id       int64              
 2   Tweet          string             
 3   Username       string             
 4   Reply_Count    int64              
 5   Retweet_Count  int64              
 6   Like_Count     int64              
 7   Bank_Search    string             
 8   Tweet_Clean    object             
 9   Sentiment      object             
 10  Hate_Speech    object             
 11  Offensive      object             
 12  Topic_Sport    object             
 13  Bank           object             
dtypes: datetime64[ns, UTC](1), int64(4), object(6), string(3)
memory usage: 504.9+ MB


In [16]:
# print duplicates based on Tweet_Id
df[['Tweet_Id', 'Bank']][df.duplicated(subset=['Tweet_Id'], keep=False)]

,Tweet_Id,Bank
37,153050311525797889,fnb
37,153050311525797889,absa
38,153050110396346369,fnb
38,153050110396346369,absa
57,153027837283139584,fnb
...,...,...
4469874,641162604120014848,standard_bank
4469888,606058966485975040,standard_bank
4469888,606058966485975040,standard_bank
4469894,598489390407622659,standard_bank


In [17]:
# generate df_hashtags
df_hashtags = generate_df_hashtags(df)
df_hashtags.head()

,Tweet_Id,hashtag_0,hashtag_1,hashtag_2,hashtag_3,hashtag_4,hashtag_5,hashtag_6,hashtag_7,hashtag_8,...,hashtag_22,hashtag_23,hashtag_24,hashtag_25,hashtag_26,hashtag_27,hashtag_28,hashtag_29,hashtag_30,Bank
0,153261300485591041,,,,,,,,,,...,,,,,,,,,,fnb
1,153254073477513216,,,,,,,,,,...,,,,,,,,,,fnb
2,153232131773038592,madlove,FNB,thinking,,,,,,,...,,,,,,,,,,fnb
3,153219883688136704,,,,,,,,,,...,,,,,,,,,,fnb
4,153203815078825984,FNB,,,,,,,,,...,,,,,,,,,,fnb


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4411950 entries, 0 to 4469910
Data columns (total 14 columns):
 #   Column         Dtype              
---  ------         -----              
 0   Datetime       datetime64[ns, UTC]
 1   Tweet_Id       int64              
 2   Tweet          string             
 3   Username       string             
 4   Reply_Count    int64              
 5   Retweet_Count  int64              
 6   Like_Count     int64              
 7   Bank_Search    string             
 8   Tweet_Clean    object             
 9   Sentiment      object             
 10  Hate_Speech    object             
 11  Offensive      object             
 12  Topic_Sport    object             
 13  Bank           object             
dtypes: datetime64[ns, UTC](1), int64(4), object(6), string(3)
memory usage: 504.9+ MB


In [19]:
# drop Tweet column
df.drop(columns=['Tweet', 'Bank_Search'], inplace=True)
# Put Tweet_Clean to the end of the dataframe
df = df[['Datetime', 'Tweet_Id', 'Username', 'Retweet_Count', 'Reply_Count', 'Like_Count', 'Sentiment', 'Hate_Speech', 'Offensive', 'Topic_Sport', 'Tweet_Clean']]

In [20]:
# set the dtypes of the columns
df['Datetime'] = pd.to_datetime(df['Datetime'])
df['Tweet_Id'] = df['Tweet_Id'].astype('int64')
df['Username'] = df['Username'].astype('string')
df['Retweet_Count'] = df['Retweet_Count'].astype('int64')
df['Reply_Count'] = df['Reply_Count'].astype('int64')
df['Like_Count'] = df['Like_Count'].astype('int64')
df['Sentiment'] = df['Sentiment'].astype('string')
df['Hate_Speech'] = df['Hate_Speech'].astype('string')
df['Offensive'] = df['Offensive'].astype('string')
df['Topic_Sport'] = df['Topic_Sport'].astype('string')
df['Tweet_Clean'] = df['Tweet_Clean'].astype('string')

In [21]:
# df and df_hashtags using parquet
df.to_parquet('Analysis_results/df_to_splitter.parquet')
df_hashtags.to_parquet('Analysis_results/df_hashtags_to_splitter.parquet')